In [12]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import autosklearn.classification
import os
import featuretools as ft


In [24]:
def run_as(df, target_ft):
    try:
        os.remove('/tmp/autosklearn_regression_example_tmp')
        os.remove('/tmp/autosklearn_regression_example_out')
    except:
        pass
    X = df.drop(columns=target_ft)
    y = df[target_ft]
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=1)
    automl = autosklearn.classification.AutoSklearnClassifier(
        time_left_for_this_task=30,
        per_run_time_limit=30,
        tmp_folder='./tmp/autosklearn_regression_example_tmp',
        output_folder='./tmp/autosklearn_regression_example_out',
    )
    automl.fit(X_train, y_train)
    y_hat = automl.predict(X_test)
    print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat))
    
def gen_feats(df):
    es = ft.EntitySet(id = 'df')
    es.entity_from_dataframe(entity_id = 'data', dataframe = df, 
                         make_index = True, index = 'index')
    feature_matrix, feature_defs = ft.dfs(entityset = es, target_entity = 'data',
                                      trans_primitives = ['add_numeric', 'multiply_numeric'])
    return feature_matrix

def run_test(df,target_ft, featools = False):
    print("Featuretools: ",featools)
    if featools:
        df = gen_feats(df)
    run_as(df,target_ft)

In [26]:
df = pd.read_csv("blood.csv")
target_ft = "class"
run_test(df, target_ft, featools=True)

Featuretools:  True
[WARNING] [2020-07-12 22:52:36,241:AutoML(1):a53eb68b3039381535b37d5d93165fbf] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (29.592487)
[WARNING] [2020-07-12 22:52:36,244:AutoML(1):a53eb68b3039381535b37d5d93165fbf] Capping the per_run_time_limit to 14.0 to have time for a least 2 models in each process.
[WARNING] [2020-07-12 22:52:55,201:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.
Accuracy score 1.0
